# WorkFlow 1: Load WebSite content into Vector DB

1.   Use LangChain LCEL
2.  Prompting + LCEL + Output Parser
3.  RAG (build once, re-use) with sources




In [ ]:
#Map Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%pip install --upgrade jupyter-client

In [ ]:
%pip install -qU \
    "requests" \
    "langchain" \
    "langchain-openai" \
    "langchain-community" \
    "langchain-text-splitters" \
    beautifulsoup4 lxml faiss-cpu langchainhub tavily-python "gradio"

In [ ]:
import importlib
def _ver(name):
    try:
        m = importlib.import_module(name)
        return getattr(m, "__version__", "n/a")
    except Exception as e:
        return f"not installed ({e})"
print("langchain           :", _ver("langchain"))
print("langgraph           :", _ver("langgraph"))
print("langchain-core      :", _ver("langchain_core"))
print("langchain-community :", _ver("langchain_community"))
print("langchain-openai    :", _ver("langchain_openai"))
print("langchainhub        :", _ver("langchainhub"))
print("langchain-text-splitters:", _ver("langchain_text_splitters"))
print("faiss-cpu           :", _ver("faiss"))
print("tavily-python       :", _ver("tavily"))

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Load URL content

In [ ]:
# RAG (v1): Web loader → splitter → FAISS → retriever → LCEL chain
#import os
#os.environ.setdefault("USER_AGENT", "IK-LangChain-RAG/1.0 (contact: ops@your-org)")  # fixes the warning

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1) Load docs (pick any public pages you want indexed)
urls = [
    "https://python.langchain.com/docs/get_started/introduction/",
    "https://docs.smith.langchain.com/"
]
loader = WebBaseLoader(urls)
docs = loader.load()
print(docs)

#Chunking

In [ ]:
# 2) Chunk using LangChain recursive splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = splitter.split_documents(docs)

print(chunks)


Generate Open AI embeddings, store in in-memory database and create retreiver object for similarity search

In [ ]:
# 3) Embed & index
emb = OpenAIEmbeddings()  # uses OPENAI_API_KEY from env
#vector database chromadb, FAISS , OPENSERACH ,
vs = FAISS.from_documents(chunks, emb)
retriever = vs.as_retriever(search_kwargs={"k": 4}) #return 4 semantically close results
print(retriever)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a precise assistant. Use the provided CONTEXT to answer.\n"
     "If the answer isn't in the context, say you don't know.\n\nCONTEXT:\n{context}"),
    ("human", "{question}")
])